## [Day 14](https://adventofcode.com/2020/day/14)

This problem has some method of storing binary numbers in which we force particular digits to remain as certain values. My understanding is that the mask. We also have different addresses for which the the data is supposed to be stored.

In [1]:
import pandas as pd
import numpy as np
import re
instr = open('../inputs/d14.txt').read().splitlines()
instr[:5]

['mask = X111000X0101100001000000100011X0000X',
 'mem[4812] = 133322396',
 'mem[39136] = 1924962',
 'mem[35697] = 29912136',
 'mem[41065] = 2558851']

So maybe I'll go back to using pandas to get this done although it seems more complicated (as usual)

In [2]:
masks = ['' if x.find('mask') == -1 else x[7:] for x in instr]

masks[:5]

['X111000X0101100001000000100011X0000X', '', '', '', '']

In [3]:
addresses, values = [], []

def process_instr(x):
    m = re.match(r"mem\[(?P<address>\d+)\] = (?P<value>\d+)", x)
    if m:
        addresses.append(m.group('address'))
        values.append(m.group('value'))
    else:
        addresses.append('')
        values.append('')

for i in range(len(instr)):
    process_instr(instr[i])
addresses[:5]

['', '4812', '39136', '35697', '41065']

In [4]:
values[:5]

['', '133322396', '1924962', '29912136', '2558851']

In [5]:
# Now we can make a data frame out of this:
instr_df = pd.DataFrame({'address' : addresses, 'value' : values, 'bit_mask' : masks})
instr_df.bit_mask = instr_df.bit_mask.replace(to_replace = '', value = None).ffill()
instr_df = instr_df.query("address != ''")
instr_df.head(7)

,address,value,bit_mask
1,4812,133322396,X111000X0101100001000000100011X0000X
2,39136,1924962,X111000X0101100001000000100011X0000X
3,35697,29912136,X111000X0101100001000000100011X0000X
4,41065,2558851,X111000X0101100001000000100011X0000X
6,38134,481,11001101X110000X010X01101100X1X0X001
7,53084,5470,11001101X110000X010X01101100X1X0X001
8,37619,2696,11001101X110000X010X01101100X1X0X001


Alright so that took quite some time but learned a few things. Next I think I wanna just turn all those values into padded binary strings.

In [6]:
bin_len = 36

def to_bin(x):
    x = int(x)
    output = ''
    for i in range(36):
        if x >= 2**(35-i):
            output += '1'
            x -= 2**(35-i)
        else:
            output += '0'
    return output

instr_df['value_bin'] = instr_df['value'].copy().map(to_bin)
instr_df.head(7)

,address,value,bit_mask,value_bin
1,4812,133322396,X111000X0101100001000000100011X0000X,000000000111111100100101011010011100
2,39136,1924962,X111000X0101100001000000100011X0000X,000000000000000111010101111101100010
3,35697,29912136,X111000X0101100001000000100011X0000X,000000000001110010000110110001001000
4,41065,2558851,X111000X0101100001000000100011X0000X,000000000000001001110000101110000011
6,38134,481,11001101X110000X010X01101100X1X0X001,000000000000000000000000000111100001
7,53084,5470,11001101X110000X010X01101100X1X0X001,000000000000000000000001010101011110
8,37619,2696,11001101X110000X010X01101100X1X0X001,000000000000000000000000101010001000


Next step will be to set up a dictionary for the addresses. We initialize it with zeroes I believe.

In [7]:
addresses_u = instr_df.address.unique().tolist()
zero_bin = to_bin('0')
addresses_i = [zero_bin for i in range(len(addresses_u))]
addresses_dict = dict(zip(addresses_u, addresses_i))
addresses_dict['4812']

'000000000000000000000000000000000000'

Alright, last setup is to make a function that takes in the binary value and the bit mask and then smooshes them.

In [8]:
def apply_mask(val, mask):
    # forgot that you cannot do positional assignment with
    # strings so we have to go to a list:
    val = [char for char in val]
    for i in range(len(mask)):
        if mask[i] != 'X':
            val[i] = mask[i]
    return ''.join(val)
apply_mask(to_bin('1'), instr_df.bit_mask.iloc[0])

'011100000101100001000000100011000001'

Next we iterate

In [9]:
# Note that index and row are given by position, not name here
for index, row in instr_df.iterrows():
    addresses_dict[row['address']] = apply_mask(row['value_bin'], row['bit_mask'])

In [10]:
# Then the solution comes from summing over the dictionary values in base 10
def to_10(x):
    tot = 0
    for i in range(len(x)):
        n = int(x[i])
        tot += n*2**(len(x)-i-1)
    return(tot)
to_10('1101')    

13

In [11]:
solution = 0
for val in addresses_dict.values():
    solution += to_10(val)
solution

8570568288597

Okay so that was somewhat laboreous but not that conceptually hard.... let's see what part 2 has in store

### Part 2

Ok this is wildly more difficult. THe twist on this is that instead of having the masks change the values, they modify the addresses which now should be see in binary.
In the mask, the symbols are seen as:  
* 1: forces that bit to a 1
* 0: leaves the bit alone
* X: makes the bit a wildcard meaning it can be either a zero or one
As a result of the wildcards, we will assign the new value to (frequently) more than one memory address.

So as I'm thinking about this, as a not trained CS person, I'm thinking there should be some way of grouping the addresses or forming some kinda network where we can remove addresses from consideration as we look through.

I was sort of thinking about making 36 dictionaries, each of which contain the addresses with a 0 in that position and then use this to successively remove choices. Let's start there and if my compute blows up I'll re-think it.

In [12]:
# first thing is I want a binary version of the addresses. and I want to reform the
# dictionary of stored values
instr_df['address_bin'] = instr_df['address'].map(to_bin)

addresses_u_bin = instr_df.address_bin.unique().tolist()
zero_bin = to_bin('0')
addresses_i = [zero_bin for i in range(len(addresses_u))]
addresses_dict_bin = dict(zip(addresses_u_bin, addresses_i))

instr_df.head()

,address,value,bit_mask,value_bin,address_bin
1,4812,133322396,X111000X0101100001000000100011X0000X,000000000111111100100101011010011100,000000000000000000000001001011001100
2,39136,1924962,X111000X0101100001000000100011X0000X,000000000000000111010101111101100010,000000000000000000001001100011100000
3,35697,29912136,X111000X0101100001000000100011X0000X,000000000001110010000110110001001000,000000000000000000001000101101110001
4,41065,2558851,X111000X0101100001000000100011X0000X,000000000000001001110000101110000011,000000000000000000001010000001101001
6,38134,481,11001101X110000X010X01101100X1X0X001,000000000000000000000000000111100001,000000000000000000001001010011110110


Now let's form the dictionary of digit position and addresses that have a zero at that point

In [13]:
# I'm very frustrated by the fact that the sums in binary have the zeroth power as
# the right most digit but the zero-th entry of a list or string is the left most entry.
# Think I'll form this by the position within a sstring since we already have our
# method to turn back to base 10 later.
zero_dict = {}

for address in addresses_dict_bin.keys():
    for i in range(32):
        if address[i] == '0':
            if i in zero_dict.keys():  
                zero_dict.update({i:zero_dict[i] + [address]})
            else:
                zero_dict.update({i:[address]})

len(zero_dict)

32

So I just realized something which is that these addresses as 36 bit are much, much too big for the largest address so if this performs super slowly, I'm going to truncate all the bit masks, values, and addresses down to speed it up.

Also neext o make the new version of the mask application:

In [14]:
def apply_mask2(val, mask):
    val = [char for char in val]
    for i in range(len(mask)):
        if mask[i] in ['X', '1']:
            val[i] = mask[i]
    return ''.join(val)


for index, row in instr_df.iterrows():
    # apply the mask, make a list of possible addresses:
    fuzzy_address = apply_mask2(row['address_bin'], row['bit_mask'])    
    contenders = list(addresses_dict_bin.keys())    
    # Now we go through the characters and update the contenders:
    for i in range(32):
        print('contenders:',len(contenders))
        if fuzzy_address[i] == 'X':
            next
        elif fuzzy_address[i] == '1':
            contenders = list(filter(lambda x: x not in zero_dict[i], contenders)) 
        else:
            contenders = list(filter(lambda x: x in zero_dict[i], contenders)) 
        
    # finally update those addresses values:
    for contender in contenders:
        addresses_dict_bin[contender] = row['value_bin']

contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contende

contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 

contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
cont

contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
co

contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 

...

contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0


contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
co

contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
co

contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders

contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 0
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 391
contenders: 0
contenders: 0
contenders: 

In [15]:
solution2 = 0
for val in addresses_dict_bin.values():
    solution2 += to_10(val)
solution2

0

### EXPERIMENTS

In [16]:
contenders = list(addresses_dict_bin.keys())
[add in zero_dict[1] for add in contenders]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [17]:
l = [1, 2, 3]
m = [3, 2, 5]
[x if x in l else next for x in m]

[3, 2, <function next>]

In [18]:
list(filter(lambda x: x in l, m)) 

[3, 2]

In [19]:
instr_df.head(20)

,address,value,bit_mask,value_bin,address_bin
1,4812,133322396,X111000X0101100001000000100011X0000X,000000000111111100100101011010011100,000000000000000000000001001011001100
2,39136,1924962,X111000X0101100001000000100011X0000X,000000000000000111010101111101100010,000000000000000000001001100011100000
3,35697,29912136,X111000X0101100001000000100011X0000X,000000000001110010000110110001001000,000000000000000000001000101101110001
4,41065,2558851,X111000X0101100001000000100011X0000X,000000000000001001110000101110000011,000000000000000000001010000001101001
6,38134,481,11001101X110000X010X01101100X1X0X001,000000000000000000000000000111100001,000000000000000000001001010011110110
7,53084,5470,11001101X110000X010X01101100X1X0X001,000000000000000000000001010101011110,000000000000000000001100111101011100
8,37619,2696,11001101X110000X010X01101100X1X0X001,000000000000000000000000101010001000,000000000000000000001001001011110011
9,12248,262787806,11001101X110000X010X01101100X1X0X001,000000001111101010011101001011011110,000000000000000000000010111111011000
11,47555,417868017,011011000111X1000100010X00XXX0X010X0,000000011000111010000010100011110001,000000000000000000001011100111000011
12,43020,11374336,011011000111X1000100010X00XXX0X010X0,000000000000101011011000111100000000,000000000000000000001010100000001100


In [20]:
to_bin('417868017')

'000000011000111010000010100011110001'

In [21]:
masks

['X111000X0101100001000000100011X0000X',
 '',
 '',
 '',
 '',
 '11001101X110000X010X01101100X1X0X001',
 '',
 '',
 '',
 '',
 '011011000111X1000100010X00XXX0X010X0',
 '',
 '',
 '',
 '',
 '',
 '',
 '0001X1X0111101X0000000101110101XX010',
 '',
 '',
 '',
 '110111X10111010X00000X1X10011X00X001',
 '',
 '',
 '',
 '110111XX0X100100000011011001XX0XX100',
 '',
 '',
 '',
 '',
 '',
 '',
 '1X011XXX0X110X000000111111X0010X1100',
 '',
 '',
 '',
 '',
 '000011110X1X1110100011XX010X0X0X00XX',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '01XX01X10111X0X1XX0X011111101110X000',
 '',
 '',
 '010XXXX10X100000010010010001011XX1XX',
 '',
 '',
 '',
 '',
 '',
 '000X1111011XX0100000X1011101X110XX00',
 '',
 '',
 '',
 '',
 '',
 '1100110X01101100X0001100111X000110X0',
 '',
 '',
 '',
 '',
 '01011X010010011X01011001000000X01X00',
 '',
 '',
 '',
 '00001X110011X1110XX0X011XX0X00100111',
 '',
 '',
 '',
 '',
 '',
 '',
 '0X00XXX1011100XX0100X11110X001110001',
 '',
 '',
 '',
 '0X011X11001101X00000000X10010101X000',
 '',
 '',
 '',
 '',

In [22]:
2**32

4294967296

In [23]:
instr_df.address.astype('int').max()

65485

In [24]:
def first_one(x):
    return x.find('1')

In [25]:
instr_df['first_1'] = instr_df.bit_mask.map(first_one)

In [26]:
instr_df.query('first_1 > 5')

,address,value,bit_mask,value_bin,address_bin,first_1
77,2424,16287,0X00XXX1011100XX0100X11110X001110001,000000000000000000000011111110011111,000000000000000000000000100101111000,7
78,62077,204882244,0X00XXX1011100XX0100X11110X001110001,000000001100001101100100000101000100,000000000000000000001111001001111101,7
79,51144,1552,0X00XXX1011100XX0100X11110X001110001,000000000000000000000000011000010000,000000000000000000001100011111001000,7


In [27]:
2**(32-7)

33554432